# Azure AI Search Index 생성 - 패션 추천 시스템

fashion_products.csv 데이터를 기반으로 Azure AI Search 인덱스를 생성합니다.
패션 상품 추천 시스템을 위한 검색 인덱스입니다.

## 주요 기능
- CSV 파일에서 패션 상품 데이터 읽기 (fashion_products.csv)
- Ada-002와 Large-3 임베딩 벡터 생성  
- Semantic Search 및 Reranker 지원
- 카테고리별 필터링 지원 (category)
- 가격대별 필터링 및 정렬 지원
- 브랜드별 검색 지원
- 배치 업로드로 효율적인 데이터 처리

## 데이터 구조
- **id**: 상품 고유 ID
- **title**: 상품명
- **brand**: 브랜드명  
- **category**: 카테고리
- **normal_price**: 정가
- **image_link**: 상품 이미지 링크
- **link**: 상품 페이지 링크
- **mobile_link**: 모바일 상품 페이지 링크
- **keyword**: 검색 키워드

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from tqdm import tqdm

load_dotenv(override=True)   

search_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("FASHION_INDEX_NAME")  # fashion-sample 인덱스 사용
admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_key = os.getenv("AZURE_OPENAI_KEY")
ada002_deployment = os.getenv("AZURE_OPENAI_ADA002_EMBEDDING_DEPLOYMENT")
large3_deployment = os.getenv("AZURE_OPENAI_3_LARGE_EMBEDDING_DEPLOYMENT")
gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT")

print(f"search_endpoint: {search_endpoint}")
print(f"fashion_index_name: {index_name}")
print(f"ada002_deployment: {ada002_deployment}")
print(f"large3_deployment: {large3_deployment}")

search_endpoint: https://hdfn-test-search.search.windows.net
fashion_index_name: fashion-sample
ada002_deployment: text-embedding-ada-002
large3_deployment: text-embedding-3-large


## Azure 클라이언트 설정

Azure Search와 OpenAI 클라이언트를 초기화합니다.

In [2]:
from azure.core.credentials import AzureKeyCredential

from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender
from azure.search.documents.models import VectorizedQuery,VectorFilterMode
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    HnswParameters,  
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    VectorSearchAlgorithmMetric,
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)


index_client = SearchIndexClient(endpoint=search_endpoint, credential=AzureKeyCredential(admin_key))
# indexer_client = SearchIndexerClient(endpoint=se, credential=AzureKeyCredential(account_key))

## 인덱스 스키마 정의

fashion_products.csv 파일의 구조에 맞는 검색 인덱스를 정의합니다.
패션 상품 추천 시스템을 위한 필드 구성입니다.

### 주요 필드
- **id**: 상품 고유 식별자 (키 필드)
- **title**: 상품명 (검색 가능, 필터링 가능)
- **brand**: 브랜드명 (검색 가능, 필터링 가능)  
- **category**: 카테고리 (검색 가능, 필터링 가능)
- **normal_price**: 정가 (정렬/필터링 가능)
- **image_link, link, mobile_link**: 상품 링크 정보
- **keyword**: 검색 키워드
- **main_text**: 통합 검색 텍스트 (title + brand + category + keyword)
- **product_group_code**: 카테고리 기반 필터링 코드
- **main_text_vector**: Ada-002 임베딩 (1536차원)
- **main_text_vector_3**: Large-3 임베딩 (3072차원)

### 벡터 검색 구성
- HNSW 알고리즘 사용 (빠른 검색)
- 코사인 유사도 기반 거리 측정
- Semantic Search 구성으로 의미 기반 검색 지원

In [3]:
# 필드명 정의 - fashion_products.csv 데이터에 맞게 수정
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String, filterable=True, facetable=True),
    SearchableField(name="brand", type=SearchFieldDataType.String, filterable=True, facetable=True),
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True, facetable=True),
    SimpleField(name="normal_price", type=SearchFieldDataType.Int64, filterable=True, facetable=True, sortable=True),
    SimpleField(name="image_link", type=SearchFieldDataType.String, filterable=False, facetable=False),
    SimpleField(name="link", type=SearchFieldDataType.String, filterable=False, facetable=False),
    SimpleField(name="mobile_link", type=SearchFieldDataType.String, filterable=False, facetable=False),
    SearchableField(name="keyword", type=SearchFieldDataType.String, filterable=True, facetable=True),
    # 검색용 통합 텍스트 필드 (title + brand + category + keyword)
    SearchableField(name="main_text", type=SearchFieldDataType.String),
    # 카테고리별 필터링을 위한 product_group_code (category를 기반으로 생성)
    SearchableField(name="product_group_code", type=SearchFieldDataType.String, filterable=True, facetable=True),
    # 가격대 코드 (추천 시스템에서 가격대별 필터링용)
    SimpleField(name="price_range", type=SearchFieldDataType.String, filterable=True, facetable=True),
    # 임베딩 벡터 필드들
    SearchField(name="main_text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
    SearchField(name="main_text_vector_3", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=3072, vector_search_profile_name="myHnswProfile")
]

# 백터 서치 configuration ( HNSW, KNN 사용 가능 )
# Option 1: 균형 잡힌 구성 - 브랜드와 카테고리 필드를 우선순위로 추가
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[
            SemanticField(field_name="main_text")   ,   # 통합텍스트
            SemanticField(field_name="title")     ,     # 상품명(중복이지만 가중치 업)
        ],
        keywords_fields=[
            SemanticField(field_name="keyword")  ,
            SemanticField(field_name="brand")    ,
            SemanticField(field_name="category") 
        ]
    )
)
semantic_search = SemanticSearch(configurations=[semantic_config])

# 자동완성을 위한 Suggester 설정
suggesters = [
    {
        "name": "sg",
        "searchMode": "analyzingInfixMatching",
        "sourceFields": ["brand", "title","category","keyword"]
    }
]

vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        )
    ]
)

try:
    # 인덱스가 이미 존재하는지 확인
    try:
        existing_index = index_client.get_index(index_name)
        print(f"기존 인덱스 '{index_name}'를 발견했습니다. 삭제 후 재생성합니다.")
        
        # 기존 인덱스 삭제
        index_client.delete_index(index_name)
        print(f"기존 인덱스 '{index_name}' 삭제 완료")
        
        # 삭제 완료 대기
        import time
        print("인덱스 삭제 완료를 기다리는 중...")
        time.sleep(5)
        
    except Exception as get_error:
        if "not found" in str(get_error).lower() or "No index with the name" in str(get_error):
            print(f"인덱스 '{index_name}'가 존재하지 않습니다. 새 인덱스를 생성합니다.")
        else:
            print(f"인덱스 확인 중 예상치 못한 오류: {get_error}")
            print("그래도 새 인덱스 생성을 시도합니다.")
    
    # 새 인덱스 생성
    print(f"새로운 패션 인덱스 '{index_name}'를 생성하는 중...")
    index = SearchIndex(
        name=index_name, 
        fields=fields, 
        vector_search=vector_search, 
        semantic_search=semantic_search, 
        suggesters=suggesters
    )
    result = index_client.create_or_update_index(index)
    print(f"새 패션 인덱스 '{result.name}' 생성 완료!")

except Exception as e:
    print(f"인덱스 생성 중 오류 발생: {e}")
    print("다시 시도하거나 Azure Portal에서 인덱스 상태를 확인해주세요.")
    raise

기존 인덱스 'fashion-sample'를 발견했습니다. 삭제 후 재생성합니다.
기존 인덱스 'fashion-sample' 삭제 완료
인덱스 삭제 완료를 기다리는 중...
새로운 패션 인덱스 'fashion-sample'를 생성하는 중...
새 패션 인덱스 'fashion-sample' 생성 완료!


## 데이터 처리 및 인덱스 업로드

fashion_products.csv 파일에서 데이터를 읽어 임베딩을 생성하고 인덱스에 업로드합니다.

### 처리 과정
1. fashion_products.csv 파일 읽기 (패션 상품 데이터)
2. 각 상품에 대해 통합 텍스트 생성 (main_text)
3. Ada-002와 Large-3 임베딩 생성
4. product_group_code 생성 (카테고리 기반)
5. price_range 생성 (가격대별 분류)
6. 배치 단위로 인덱스 업로드

### 가격대 분류 기준
- 1만원대: 10,000 ~ 19,999원
- 2만원대: 20,000 ~ 29,999원
- ...
- 100만원이상: 1,000,000원 이상

In [4]:
# OpenAI 클라이언트 초기화
embed_client = AzureOpenAI(
    azure_deployment=ada002_deployment,
    api_version="2023-05-15",
    azure_endpoint=openai_endpoint,
    api_key=openai_key,
)

embed3_client = AzureOpenAI(
    azure_deployment=large3_deployment,
    api_version="2023-05-15",
    azure_endpoint=openai_endpoint,
    api_key=openai_key,
)

def get_embedding_ada002(text):
    """Ada-002 임베딩 생성"""
    try:
        response = embed_client.embeddings.create(input=[text], model=ada002_deployment)
        return response.data[0].embedding
    except Exception as e:
        print(f"Ada-002 임베딩 생성 실패: {e}")
        return None

def get_embedding_large3(text):
    """Large-3 임베딩 생성"""
    try:
        response = embed3_client.embeddings.create(input=[text], model=large3_deployment)
        return response.data[0].embedding
    except Exception as e:
        print(f"Large-3 임베딩 생성 실패: {e}")
        return None

def get_price_range(price):
    """가격을 기반으로 가격대 코드 생성"""
    try:
        price = int(price)
        
        # 가격대 구간 정의 (임계값, 라벨)
        price_ranges = [
            (10000, "1만원미만"),
            (20000, "1만원대"),
            (30000, "2만원대"), 
            (40000, "3만원대"),
            (50000, "4만원대"),
            (60000, "5만원대"),
            (70000, "6만원대"),
            (80000, "7만원대"),
            (90000, "8만원대"),
            (100000, "9만원대"),
            (200000, "10만원대"),
            (300000, "20만원대"),
            (400000, "30만원대"),
            (500000, "40만원대"),
            (600000, "50만원대"),
            (700000, "60만원대"),
            (800000, "70만원대"),
            (900000, "80만원대"),
            (1000000, "90만원대")
        ]
        
        # 해당하는 가격대 찾기
        for threshold, label in price_ranges:
            if price < threshold:
                return label
        
        # 모든 구간을 초과하는 경우
        return "100만원이상"
        
    except (ValueError, TypeError):
        return "가격미상"

# CSV 파일 읽기
print("fashion_products.csv 파일을 읽는 중...")
df = pd.read_csv('fashion_products.csv')
print(f"총 {len(df)}개의 패션 상품 데이터를 로드했습니다.")

# 데이터 전처리 및 변환
documents = []
batch_size = 10  # 배치 크기 설정

print("데이터를 변환하고 임베딩을 생성하는 중...")
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing items"):
    # main_text 생성 (검색용 통합 텍스트)
    main_text_parts = [
        str(row['title']) if pd.notna(row['title']) else '',
        str(row['brand']) if pd.notna(row['brand']) else '',
        str(row['category']) if pd.notna(row['category']) else '',
        str(row['keyword']) if pd.notna(row['keyword']) else ''
    ]
    main_text = ' '.join([part for part in main_text_parts if part])
    
    # product_group_code는 category를 기반으로 생성
    product_group_code = str(row['category']).lower().replace(' ', '_').replace('/', '_') if pd.notna(row['category']) else 'unknown'
    
    # 가격대 코드 생성
    price_range = get_price_range(row['normal_price']) if pd.notna(row['normal_price']) else '가격미상'
    
    # 임베딩 생성
    ada002_embedding = get_embedding_ada002(main_text)
    large3_embedding = get_embedding_large3(main_text)
    
    if ada002_embedding is None or large3_embedding is None:
        print(f"행 {idx}에서 임베딩 생성 실패, 건너뜀")
        continue
    
    # 문서 생성
    document = {
        "id": str(row['id']),
        "title": str(row['title']) if pd.notna(row['title']) else '',
        "brand": str(row['brand']) if pd.notna(row['brand']) else '',
        "category": str(row['category']) if pd.notna(row['category']) else '',
        "normal_price": int(row['normal_price']) if pd.notna(row['normal_price']) else 0,
        "image_link": str(row['image_link']) if pd.notna(row['image_link']) else '',
        "link": str(row['link']) if pd.notna(row['link']) else '',
        "mobile_link": str(row['mobile_link']) if pd.notna(row['mobile_link']) else '',
        "keyword": str(row['keyword']) if pd.notna(row['keyword']) else '',
        "main_text": main_text,
        "product_group_code": product_group_code,
        "price_range": price_range,
        "main_text_vector": ada002_embedding,
        "main_text_vector_3": large3_embedding
    }
    
    documents.append(document)

print(f"총 {len(documents)}개의 문서가 준비되었습니다.")

# 인덱스에 문서 업로드
print("인덱스에 문서를 업로드하는 중...")
indexing_sender = SearchIndexingBufferedSender(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(admin_key)
)

# 배치로 업로드
for i in tqdm(range(0, len(documents), batch_size), desc="Uploading batches"):
    batch = documents[i:i+batch_size]
    try:
        indexing_sender.upload_documents(documents=batch)
    except Exception as e:
        print(f"배치 {i//batch_size + 1} 업로드 실패: {e}")

# 최종 플러시
indexing_sender.flush()
print(f"{len(documents)}개의 패션 상품이 성공적으로 인덱스에 업로드되었습니다.")

fashion_products.csv 파일을 읽는 중...
총 464개의 패션 상품 데이터를 로드했습니다.
데이터를 변환하고 임베딩을 생성하는 중...


Processing items: 100%|██████████| 464/464 [04:29<00:00,  1.72it/s]


총 464개의 문서가 준비되었습니다.
인덱스에 문서를 업로드하는 중...


Uploading batches: 100%|██████████| 47/47 [00:00<00:00, 17568.16it/s]


464개의 패션 상품이 성공적으로 인덱스에 업로드되었습니다.


## 인덱스 확인

업로드된 데이터를 확인합니다.

In [11]:
# 인덱스 정보 확인
search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=AzureKeyCredential(admin_key))

# 인덱스 통계 확인
try:
    # 간단한 검색으로 문서 수 확인
    results = search_client.search(search_text="*", top=1, include_total_count=True)
    total_count = results.get_count()
    print(f"인덱스에 저장된 총 문서 수: {total_count}")
    
    # 샘플 문서 확인
    sample_results = search_client.search(search_text="*", top=3)
    print("\n샘플 문서:")
    for i, result in enumerate(sample_results, 1):
        print(f"\n{i}. ID: {result.get('id', 'N/A')}")
        print(f"   Title: {result.get('title', 'N/A')}")
        print(f"   Brand: {result.get('brand', 'N/A')}")
        print(f"   Category: {result.get('category', 'N/A')}")
        print(f"   Price: {result.get('normal_price', 'N/A'):,}원")
        print(f"   Price Range: {result.get('price_range', 'N/A')}")
        print(f"   Product Group Code: {result.get('product_group_code', 'N/A')}")
        
    # 카테고리별 통계 확인
    print("\n=== 카테고리별 상품 수 ===")
    category_stats = search_client.search(
        search_text="*", 
        facets=["category,count:20"], 
        top=0
    )
    
    if hasattr(category_stats, 'get_facets') and category_stats.get_facets():
        for facet_result in category_stats.get_facets()['category']:
            print(f"{facet_result['value']}: {facet_result['count']}개")
    
    # 가격대별 통계 확인  
    print("\n=== 가격대별 상품 수 ===")
    price_stats = search_client.search(
        search_text="*", 
        facets=["price_range,count:20"], 
        top=0
    )
    
    if hasattr(price_stats, 'get_facets') and price_stats.get_facets():
        for facet_result in price_stats.get_facets()['price_range']:
            print(f"{facet_result['value']}: {facet_result['count']}개")
        
except Exception as e:
    print(f"인덱스 확인 중 오류 발생: {e}")

print("\n🎉 패션 상품 추천 시스템 인덱스 생성이 완료되었습니다!")


인덱스에 저장된 총 문서 수: 464

샘플 문서:

1. ID: 40A2088715
   Title: [창주스토어 더현대대구][CC콜렉트] C252PST051 스트라이프 스퀘어넥 탑 
   Brand: CC콜렉트(백화점정상)
   Category: 패션의류
   Price: 99,000원
   Price Range: 9만원대
   Product Group Code: 패션의류

2. ID: 40A2157657
   Title: [창주스토어 무역센터점][Mare di Mari] Stripe Round Neck Top - Ivory 
   Brand: 마레디마리
   Category: 패션의류
   Price: 223,200원
   Price Range: 20만원대
   Product Group Code: 패션의류

3. ID: 60A1925398
   Title: [창주스토어 무역센터점][리이/RE RHEE] BLUR LOGO ORGANIC COTTON T SHIRT 
   Brand: 리이
   Category: 패션의류
   Price: 50,460원
   Price Range: 5만원대
   Product Group Code: 패션의류

=== 카테고리별 상품 수 ===
패션의류: 464개

=== 가격대별 상품 수 ===
10만원대: 136개
5만원대: 66개
8만원대: 51개
20만원대: 39개
7만원대: 39개
4만원대: 38개
3만원대: 31개
6만원대: 26개
9만원대: 23개
2만원대: 7개
30만원대: 5개
40만원대: 2개
1만원대: 1개

🎉 패션 상품 추천 시스템 인덱스 생성이 완료되었습니다!
